In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install keras_applications 

     |████████████████████████████████| 50 kB 2.9 MB/s 


In [3]:
import pandas as pd
import numpy as np
import os.path
import matplotlib.pyplot as plt
import seaborn as sns
import gc

from PIL import Image

%matplotlib inline
from tensorflow import keras 
from tensorflow.keras.applications import ResNet50,ResNet101
import cv2
from tqdm import tqdm
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import backend as K
from tensorflow.keras import applications
from tensorflow.keras.models import Model
from tensorflow.keras import optimizers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing import image

In [4]:
from sklearn.model_selection import StratifiedKFold , KFold ,RepeatedKFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

In [5]:
import keras
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, MaxPool2D
from keras.layers.core import Lambda
from tensorflow.keras.utils import to_categorical
from keras.preprocessing import image
from tensorflow.keras import datasets, layers, models, losses
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm import tqdm
%matplotlib inline
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import *
#from keras.utils import multi_gpu_model
from keras.models import load_model
import keras.backend as K
from keras.layers.core import Lambda
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.applications.inception_v3 import InceptionV3 
from keras.applications.densenet import DenseNet121
from keras.applications.vgg16 import VGG16
#from keras.applications.resnet50 import ResNet50
#from keras_applications.resnet import ResNet50
from keras.applications.xception import Xception
from keras.applications.mobilenet import MobileNet
import os
from keras.layers import *
from sklearn.model_selection import KFold, StratifiedKFold

In [6]:
def get_base_model(model_name,image_size):
    if model_name =='vgg16':
        base_model=VGG16              (include_top=False,weights='imagenet',input_shape=(image_size,image_size,3))
    if model_name =='resnet50':
        base_model=ResNet50           (include_top=False,weights='imagenet',input_shape=(image_size,image_size,3))
    if model_name =='xception':
        base_model=Xception           (include_top=False, weights='imagenet',input_shape=(image_size,image_size,3))
    if model_name =='densenet121':    #done acc = 55% epochs:30
        base_model=DenseNet121       (include_top=False, weights='imagenet',input_shape=(image_size,image_size,3))
    if model_name =='mobilenet0.75': #acc = 60.41% loss = 1.21 epochs:70
        base_model=MobileNet         (include_top=False,weights='imagenet',alpha=0.75,input_shape=(image_size,image_size,3))
    if model_name =='mobilenet1.0': #acc = 60.41% loss = 1.21 epochs:70
        base_model=MobileNet         (include_top=False,weights='imagenet',alpha=1.0,input_shape=(image_size,image_size,3))
    if model_name =='mobilenetv2':
        base_model=MobileNetV2      (include_top=False,weights='imagenet',alpha=1.0,input_shape=(image_size,image_size,3))
    if model_name =='inceptionv3':   
        base_model=InceptionV3       (include_top=False,weights='imagenet',input_shape=(image_size,image_size,3))
    if model_name =='inceptionv2':
        base_model=tf.keras.application.InceptionResNetV2 (include_top=False, weights='imagenet',input_shape=(image_size,image_size,3))
    return base_model

def Global_attention_block(inputs):
    shape=K.int_shape(inputs)
    
    x=tf.keras.layers.AveragePooling2D(pool_size=(shape[1],shape[2])) (inputs)
    x=Conv2D(shape[3],1, padding='same') (x)
    x=Activation('relu') (x)

    x=Conv2D(shape[3],1, padding='same') (x)
    x=Activation('sigmoid') (x)
    C_A=tf.keras.layers.Multiply()([x,inputs])
    
    x=Lambda(lambda x: K.mean(x,axis=-1,keepdims=True)) (C_A)
    x=Activation('sigmoid') (x)
    S_A=tf.keras.layers.Multiply()([x,C_A])
    return S_A

def Category_attention_block(inputs,classes,k):
    shape=K.int_shape(inputs)
    F=Conv2D(k*classes,1, padding='same') (inputs)
    F=tf.keras.layers.BatchNormalization() (F)
    F1=Activation('relu') (F)
    
    F2=F1
    x=tf.keras.layers.GlobalMaxPool2D()(F2)
    
    x=tf.keras.layers.Reshape((classes,k)) (x)
    S=Lambda(lambda x: K.mean(x,axis=-1,keepdims=False))  (x)
    
    x=tf.keras.layers.Reshape((shape[1],shape[2],classes,k)) (F1)
    x=Lambda(lambda x: K.mean(x,axis=-1,keepdims=False))  (x)
    x=tf.keras.layers.Multiply()([S,x])
    M=Lambda(lambda x: K.mean(x,axis=-1,keepdims=True))  (x)
    
    semantic=tf.keras.layers.Multiply()([inputs,M])
    return semantic


k=3
batch_size= 16
image_size=224
classes=2
Epochs1 = 1
lr1=0.005
lr2=0.0001
train_num = 1200
loss_fun= 'categorical_crossentropy' 

base_model=get_base_model('densenet121',image_size)  
base_in=base_model.input
base_out=base_model.output

shape = K.int_shape(base_out)
channel_val = shape[3]/2

red_feat = tf.keras.layers.Conv2D(channel_val,1,padding='same')(base_out)

x=Global_attention_block(red_feat)
base_out=Category_attention_block(x,classes,k)

shape=K.int_shape(base_out)  
x=GlobalAveragePooling2D()(base_out)
out=Dense(classes,activation='softmax')(x)

parallel_model=keras.Model(base_model.input,out)

train_dir = '../content/drive/MyDrive/Mtech Project/Dataset/Messidor_10_fold/images/'
train=ImageDataGenerator(horizontal_flip=True,vertical_flip=True,rotation_range=90)          
train_data=train.flow_from_directory(train_dir,
                                     target_size=(image_size,image_size),
                                     shuffle = True,
                                     batch_size=batch_size)

parallel_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr1,decay=0.00001),loss=loss_fun,metrics=['acc'])

lr_decay=ReduceLROnPlateau(monitor='loss', factor=0.8, patience=3, verbose=1)

    
filepath = "k_fold_densenet121_512_red_cab_messidor_weights.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='acc',verbose=1, save_best_only=True, mode='max')

for layer in base_model.layers:
  layer.trainable = False   
        
parallel_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr1,decay=0.00001),loss=loss_fun,metrics=['acc'])

parallel_model.fit(train_data,
                   steps_per_epoch=train_num/batch_size,
                   epochs=Epochs1, 
                   workers=2,
                   callbacks=[lr_decay,checkpoint])   
    
for layer in parallel_model.layers:
        layer.trainable = True
        
parallel_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr2,decay=0.00001),loss=loss_fun,metrics=['acc'])

29097984/29084464 [==============================] - 0s 0us/step
Found 1200 images belonging to 2 classes.
75/75 [==============================] - ETA: 0s - loss: 1.3064 - acc: 0.5267
Epoch 1: acc improved from -inf to 0.52667, saving model to k_fold_densenet121_512_red_cab_messidor_weights.hdf5
75/75 [==============================] - 80s 787ms/step - loss: 1.3064 - acc: 0.5267 - lr: 0.0050


In [7]:
train = pd.read_csv('/content/drive/MyDrive/Mtech Project/Dataset/Messidor_10_fold/binary_all_data_table_messidor_string.csv')
TRAIN_PATH = '../content/drive/MyDrive/Mtech Project/Dataset/Messidor_10_fold/all_img/messidor_all_image'
train.head()

,Image,ID
0,20060410_39229_0200_PP.tif,Class_zero
1,20051216_45992_0200_PP.tif,Class_zero
2,20051021_59589_0100_PP.tif,Class_zero
3,20060407_44400_0200_PP.tif,Class_zero
4,20060407_44042_0200_PP.tif,Class_zero


In [8]:
# Increasing the size of dataset without disturbing their corresponding ratios 
main_pred = []
error = []
data_kfold = pd.DataFrame()

df = train.copy()
class_0 = train[train["ID"]== 'Class_zero']
class_1 = train[train["ID"]== 'Class_one']

#df = pd.concat([df,class_0])
#df = pd.concat([df,class_1])

train_y = df.ID
train_x = df.drop(['ID'],axis=1)

In [10]:
#Initializing Data Generators
train_datagen = ImageDataGenerator(horizontal_flip=True,vertical_flip=True,rotation_range=90) 
validation_datagen = ImageDataGenerator()
Epochs2 = 15
splits = 10

# k-fold
kfold = StratifiedKFold(n_splits=splits,shuffle=True,random_state=42)
j = 0

# K-fold Train and test for each split
for train_idx, val_idx in list(kfold.split(train_x,train_y)):
    x_train_df = df.iloc[train_idx]  
    x_valid_df = df.iloc[val_idx]

    j+=1
    print('Fold : {}'.format(j))
    training_set = train_datagen.flow_from_dataframe(dataframe=x_train_df, directory=TRAIN_PATH,
                                                 x_col="Image", y_col="ID",
                                                 class_mode="categorical",
                                                 target_size=(image_size,image_size), batch_size=batch_size)
    
    validation_set = validation_datagen.flow_from_dataframe(dataframe=x_valid_df, directory=TRAIN_PATH,
                                                 x_col="Image", y_col="ID",
                                                 class_mode="categorical",
                                                 target_size=(image_size,image_size), batch_size=batch_size)
         
    history = parallel_model.fit(training_set,
                                 #validation_data=validation_set,
                                 epochs = Epochs2,
                                 steps_per_epoch=x_train_df.shape[0] // batch_size,
                                 callbacks=[lr_decay,checkpoint]
                                )
    
    '''test_generator = ImageDataGenerator()
    test_set = test_generator.flow_from_dataframe(dataframe=train, directory=TRAIN_PATH,
                                                 x_col="Image",y_col=None,
                                                 class_mode=None,
                                                 target_size=(image_size,image_size))'''
    
    pred = parallel_model.predict(validation_set, (len(train)//splits) // batch_size)
    predicted_class_indices=np.argmax(pred,axis=1)
                                       
    data_kfold[j] = predicted_class_indices
    gc.collect()
    print('----------------------------------------------------------------------------')

Fold : 1
Found 1080 validated image filenames belonging to 2 classes.
Found 120 validated image filenames belonging to 2 classes.
Epoch 1/15
67/67 [==============================] - ETA: 0s - loss: 0.5754 - acc: 0.7472
Epoch 1: acc improved from 0.52667 to 0.74718, saving model to k_fold_densenet121_512_red_cab_messidor_weights.hdf5
67/67 [==============================] - 121s 2s/step - loss: 0.5754 - acc: 0.7472 - lr: 1.0000e-04
Epoch 2/15
67/67 [==============================] - ETA: 0s - loss: 0.5448 - acc: 0.7660
Epoch 2: acc improved from 0.74718 to 0.76598, saving model to k_fold_densenet121_512_red_cab_messidor_weights.hdf5
67/67 [==============================] - 30s 437ms/step - loss: 0.5448 - acc: 0.7660 - lr: 1.0000e-04
Epoch 3/15
67/67 [==============================] - ETA: 0s - loss: 0.5318 - acc: 0.7914
Epoch 3: acc improved from 0.76598 to 0.79135, saving model to k_fold_densenet121_512_red_cab_messidor_weights.hdf5
67/67 [==============================] - 30s 436ms/st

In [11]:
print(pred)

[[2.1759389e-11 1.0000000e+00]
 [6.8656182e-01 3.1343815e-01]
 [1.2116940e-17 1.0000000e+00]
 [6.8656182e-01 3.1343815e-01]
 [6.8656182e-01 3.1343815e-01]
 [6.8654412e-01 3.1345591e-01]
 [6.8656182e-01 3.1343815e-01]
 [8.0366147e-10 1.0000000e+00]
 [1.5168830e-06 9.9999845e-01]
 [6.8656182e-01 3.1343815e-01]
 [5.0056434e-07 9.9999952e-01]
 [6.8656182e-01 3.1343815e-01]
 [1.4430943e-10 1.0000000e+00]
 [8.4563308e-13 1.0000000e+00]
 [6.0856443e-08 9.9999988e-01]
 [6.8656182e-01 3.1343815e-01]
 [1.6401515e-16 1.0000000e+00]
 [1.6145026e-07 9.9999988e-01]
 [4.2956412e-09 1.0000000e+00]
 [5.1219957e-09 1.0000000e+00]
 [6.8649584e-01 3.1350416e-01]
 [6.8656182e-01 3.1343815e-01]
 [6.8656182e-01 3.1343815e-01]
 [6.8656182e-01 3.1343815e-01]
 [6.8656182e-01 3.1343815e-01]
 [2.8633142e-06 9.9999714e-01]
 [7.2657581e-06 9.9999273e-01]
 [6.8656182e-01 3.1343815e-01]
 [6.8556136e-01 3.1443867e-01]
 [6.8656182e-01 3.1343815e-01]
 [6.8656182e-01 3.1343815e-01]
 [4.3259517e-11 1.0000000e+00]
 [6.8656

In [12]:
print(predicted_class_indices)

[1 0 1 0 0 0 0 1 1 0 1 0 1 1 1 0 1 1 1 1 0 0 0 0 0 1 1 0 0 0 0 1 0 1 1 0 1
 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 0 0 1 0 0 0 0 1 1 1 0 1 0 1 0 1 1 1 0 0 1 0
 0 0 0 1 0 1 1 1 1 1 0 1 0 1 0 0 1 1 0 0 0 0 1 1 0 1 0 0 1 1 1 1 1 0 1 1 1
 1 1 1 1 1 1 0 0 0]


In [13]:
# holder
data_kfold

,1,2,3,4,5,6,7,8,9,10
0,1,1,0,1,1,1,0,1,1,1
1,1,0,1,1,0,1,0,0,0,0
2,1,1,1,0,0,0,1,1,0,1
3,1,0,1,1,0,0,0,0,1,0
4,0,1,1,0,1,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...
115,0,0,1,0,1,0,1,1,1,1
116,0,1,0,1,1,0,1,0,1,1
117,1,0,1,1,0,0,1,1,0,0
118,1,1,1,0,0,1,1,0,1,0


In [14]:
data_kfold[4]

0      1
1      1
2      0
3      1
4      0
      ..
115    0
116    1
117    1
118    0
119    0
Name: 4, Length: 120, dtype: int64

In [15]:
# testing on single iteration of HoldOut
predicted_class_indices = data_kfold[4]
labels=(training_set.class_indices)
labels2=dict((v,k) for k,v in labels.items())
predictions=[labels2[k] for k in predicted_class_indices]

In [16]:
print(labels)
print("----------------------------------------------------------")
print(labels2)
print("----------------------------------------------------------")
print(predictions)

{'Class_one': 0, 'Class_zero': 1}
----------------------------------------------------------
{0: 'Class_one', 1: 'Class_zero'}
----------------------------------------------------------
['Class_zero', 'Class_zero', 'Class_one', 'Class_zero', 'Class_one', 'Class_one', 'Class_zero', 'Class_zero', 'Class_zero', 'Class_zero', 'Class_zero', 'Class_one', 'Class_zero', 'Class_one', 'Class_one', 'Class_one', 'Class_zero', 'Class_zero', 'Class_zero', 'Class_one', 'Class_zero', 'Class_zero', 'Class_one', 'Class_zero', 'Class_zero', 'Class_one', 'Class_zero', 'Class_zero', 'Class_one', 'Class_one', 'Class_one', 'Class_zero', 'Class_one', 'Class_zero', 'Class_zero', 'Class_one', 'Class_zero', 'Class_zero', 'Class_zero', 'Class_zero', 'Class_one', 'Class_one', 'Class_zero', 'Class_zero', 'Class_one', 'Class_zero', 'Class_one', 'Class_zero', 'Class_one', 'Class_zero', 'Class_one', 'Class_one', 'Class_zero', 'Class_zero', 'Class_zero', 'Class_one', 'Class_one', 'Class_zero', 'Class_zero', 'Class_zero

In [ ]:
#print("Accuracy of CABNet K-fold: ",accuracy_score(predictions,train.ID))

In [17]:
ans = train.copy()

In [18]:
# Taking The Label with Maximum Occurences

labels=(training_set.class_indices)
labels2=dict((v,k) for k,v in labels.items())
import collections 
for i in range(len(data_kfold)):
    co = collections.Counter(data_kfold.loc[i])
    co = sorted(co.items(),key=lambda x: x[1],reverse=True)
    ans.ID.loc[i] = labels2[co[0][0]]

In [19]:
# Averaged K-Fold Output
print("Accuracy of K-Fold Method: ",accuracy_score(train.ID,ans.ID))

Accuracy of K-Fold Method:  0.9716666666666667
